# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM이 외부 데이터를 컨텍스트로 활용.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
# %pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# x = embedding.embed_query('점심 배불리 먹었더니 졸리다..')
# print(len(x), x)

# VectorStore 에 임베딩 후 저장(In memory)
vectorstore = FAISS.from_documents(documents, embedding=embedding)

In [ ]:
vectorstore.similarity_search("초보자가 키우기 좋은 애완동물 추천해줘", k=2)

[Document(id='07b8f509-f283-4657-bf6e-37141777a8b1', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='134681fc-ce11-4939-b981-c389dcefa781', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [6]:
# %pip install -q pymupdf

In [ ]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./data/spri.pdf")
docs = loader.load()


print("원본 pdf의 장수", len(docs))

원본 pdf의 장수 23


In [ ]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print("분할 후 청크 수", len(split_docs))

분할 후 청크 수 72


In [ ]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search("미국 대통령과 관련된 문서들 가져와봐", k=4)

[Document(id='f6463710-2d0c-4d71-9d5f-33ebd6d6a10b', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM 
4. 최종 답변

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅 (랭체인 허브에서 가져오기)
prompt = hub.pull("rlm/rag-prompt")

# LLM 모델
llm = ChatOpenAI(model="gpt-5-nano")

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("삼성전자 관련 소식 다 가져와")

'삼성전자는 2023년 삼성 AI 포럼에서 자체 개발 생성 AI 모델 ‘삼성 가우스’를 공개했고, 언어, 코드, 이미지의 3개 모델로 구성되어 온디바이스에서 작동하도록 설계됐다.  \n가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터로 학습되었으며 외부로 사용자 정보가 유출되지 않는 점을 강조한다.  \n또한 코드 모델 ‘코드아이(code.i)’ 같은 AI 코딩 어시스턴트가 포함되며, 다양한 제품에 단계적으로 탑재될 계획으로 2024년 이후 가우스 탑재 기기가 늘어날 전망이다.'

In [ ]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch

from datetime import datetime

# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime("%Y-%m-%d")

llm = ChatOpenAI(model="gpt-5-nano")

search_tool = TavilySearch(max_results=5, topic="general")

loader = PyMuPDFLoader("./data/spri.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="PDF 문서에서 질문과 관련된 내용을 검색합니다.",  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", text),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # 도구(검색) 호출때 필요함
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, memory=memory, tools=[search_tool, rag_tool], verbose=True
)

In [ ]:
agent_executor.invoke(
    {"input": "방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?"}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'AI industry trends 2024 2025 report foundation models regulation safety enterprise adoption 2025', 'start_date': '2024-01-01', 'end_date': '2025-09-03', 'search_depth': 'advanced', 'topic': 'general'}`


{'query': 'AI industry trends 2024 2025 report foundation models regulation safety enterprise adoption 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://medium.com/@dejanmarkovic_53716/ai-trends-for-2025-enterprise-adoption-challenges-solutions-e4ee075788dd', 'title': 'AI Trends for 2025: Enterprise Adoption Challenges & Solutions', 'content': 'Data from Gartner indicates that by 2025, over 75% of enterprises will have moved beyond experimental AI deployments to implement production-ready systems that directly impact core business operations. This represents a significant maturation of the market from the exploratory phase that characterized the early 2020s. 

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='요약하면 2023년의 흐름과 비교해 올해(2025년)에는 AI가 더 실무에 깊이 들어가고, 규제/거버넌스도 더 강하게 자리 잡았으며, 의료 같은 분야로의 확장도 가속화되었습니다. 아래는 최근 트렌드의 핵심 포인트입니다.\n\n주요 업데이트 포인트\n- 생산적 확산(프로덕션 레디 AI)\n  - 엔터프라이즈에서 실험 단계에서 벗어나 생산 운영에 바로 적용하는 사례가 늘어나고 있습니다. 최신 분석에 따르면 2025년까지 다수의 기업이 생산 가능한 AI 시스템을 운영에 반영하고 있다고 합니다.\n\n- Foundation 모델의 주도권 확산\n  - 산업체가 주도하는 대형 모델 개발이 폭넓게 확대되었습니다. 2024년 상면의 모델 중 거의 90%가 산업계에서 나오고, 학계의 연구 기여도는 여전히 많이 높지만 모델 규모와 실용화 속도가 산업 쪽에 집중되고 있습니다.\n  - 의료 분야로의 확장도 가속화되어 Med-Gemini 같은 대형 의학 foundation 모델과 해당 분야 특화 모델들이 속속 등장하고 있습니다(의료 영상·진단 보조 등).\n\n- 투자와 데이터/컴퓨트 성장\n  - Generative AI 분야의 민간 투자 규모가 크게 증가했습니다. 2024년에는 약 339억 달러 수준으로 전년 대비 상승하고, 2022년 대비 몇 배 성장한 수치를 기록했습니다.\n  - 모델 규모의 증가 속도도 빨라지고 있습니다. 5개월마다 훈련 컴퓨트가, 8개월마다 데이터셋이 크게 증가하는 추세입니다. 다만 이와 함께 에너지 소비 증가와 같은 도전도 병행하고 있습니다.\n  - 학계보다 산업계가 혁신의 속도를 주도하는 한편, 상위 모델 간 성능